In [1]:
import ipywidgets as widgets
from IPython.display import display
import string

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from nltk.stem import PorterStemmer
ps = PorterStemmer()

from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

import sqlite3

import scipy.sparse as sparse

import numpy as np

import math

from bs4 import BeautifulSoup

In [2]:
# reading  and parsing the inv_idx.txt and doc_mag.txt files 
# .txt files should be placed in the same directory as this python file

In [3]:
inv_idx = open('inv_idx.txt','r')
word = inv_idx.readlines()
inv_idx.close()

record = []
for el in word :
    x = el.split('[')
    w = x[0][2:-3]
    tup = x[1][:-3]
    record.append((w,tup))
    

doc_mag = open('doc_mag.txt','r')
doc = doc_mag.readlines()
doc_mag.close()

record_d = []
for el in doc:
    x = el.split('(')
    did = x[1][:-2]
    mag = x[2][4:-3]
    record_d.append((did,mag))

In [4]:
# creating table for inverted_ind.txt

In [5]:
conn = sqlite3.connect('Cosine_values.db')
c = conn.cursor()

In [6]:
c.execute('create table if not exists inverted_ind(word varchar(100) PRIMARY KEY, postinglist TEXT)')
c.executemany('insert into inverted_ind(word,postinglist) values(?,?)',record)

In [7]:
# table for doc_mag.txt

In [8]:
c.execute('create table if not exists doc_mag(doc_id Integer PRIMARY KEY, magnitude Integer)')
c.executemany('insert into doc_mag(doc_id,magnitude) values(?,?)',record_d)

In [9]:
# function for computing cosine_similarity

In [10]:
def cosine_sim(query):
    try:
        s_q = query
    #     1. lower_case
        query = query.lower()

    # 2.filtering noise from data (punctuations + digits + alphanumerics)

      # noise = string.punctuation+string.digits
        noise = '!"#$%&\()*+,-:;<=>?@[\\]^_`{|}~'+string.digits
        translation = str.maketrans(string.ascii_letters, string.ascii_letters, noise)
        query = query.translate(translation)  

        trans1 = str.maketrans('\n\r\t\'/.','      ')
        query = query.translate(trans1)

    # 3. stop words 
        query_sw = [word for word in query.split(' ') if word not in stop_words]
        query = ''


    # 4.stemming/lemmatization

        for word in query_sw:
            #  temp = ps.stem(word)
            temp = lem.lemmatize(word)
            #  temp = ps.stem(temp)
            query = query+temp+' '

    # 5. list of distinct words

        query_distinct = list(dict.fromkeys(query.split(' ')))
        query_distinct.sort()

        try:
            query_distinct_2 = [word for word in query_distinct if len(word)>1]
            # query_distinct_2.remove('')
        except:
            print('no strings < length 2')

        # count of occurrences of distinct words in doc
        count_list = []
        list_tf_query = []
        for word in query_distinct_2:
            count_list.append(query.split(' ').count(word))

        max_count_list = max(count_list)

        tfidf_val = []
        for word in query_distinct_2:
            tfidf_val.append(query.split(' ').count(word)/(max_count_list))
            list_tf_query.append((word,(query.split(' ').count(word),(query.split(' ').count(word)/(max_count_list)))))


        doc_id_list = []
        tf_idf_list = []
        word_list = []
        mag_list = []

        cnt = 0

        try:

            for el in list_tf_query:
                word = "'"+el[0]+"'"

                c.execute("select * from inverted_ind where word="+word)
                rslt = c.fetchone()

                count = 0

                for x in rslt:
                    count = count+1
                    # tuple
                    if count%2==0:
                        temp = x.split('(')[1:]

                        for x in temp:
                            t = x.split(',')
                            did = int(t[0])
                            tf_idf = float(t[2][1:-1])
                            doc_id_list.append(did)
                            tf_idf_list.append(tf_idf)
                            word_list.append(word_cnt)

                    # word
                    else:
                        word = x
                        word_cnt = cnt
                        cnt = cnt+1

        #                 print(word)

        #     print(doc_id_list)
        #     print(tf_idf_list)
        #     print(word_list)


            # sparse matrix of docid x words

            matrix = sparse.coo_matrix((tf_idf_list,(doc_id_list,word_list)))
            matrix_csr = matrix.tocsr()
            matrix_dense = matrix.todense()


        #     numerator

        #     print(matrix_dense)
        #     print(np.array([tfidf_val]).T)
        #     print(np.dot(matrix_csr[5],np.array([tfidf_val]).T))

            cosine_mat = np.dot(matrix_dense,np.array([tfidf_val]).T)
            cosine_list = cosine_mat.tolist()

            #numerator
            cos_flat_list = [item for sublist in cosine_list for item in sublist]

            #denominator
            q_mag = 0
            for x in list_tf_query:
                y=x[1][1]
                q_mag = (y*y) + q_mag

            doc_id_s = list(dict.fromkeys(doc_id_list))
            doc_id_s.sort()


            for i in doc_id_s:
                c.execute('select magnitude from doc_mag where doc_id='+str(i))
                mag_list.append(c.fetchone()[0])

            mag_list_1=[]
            idx= 0
            for val in cos_flat_list:
                if val == float(0) :
                    mag_list_1.append(0)
                else:
                    mag_list_1.append(q_mag*mag_list[idx])
                    idx = idx + 1


        #     print(cos_flat_list)
        #     print(mag_list_1)    

            cos_list = []
            for x in range(len(cos_flat_list)):
                if mag_list_1[x]== float(0):
                    cos_list.append(0)
                else:
                    cos_list.append((cos_flat_list[x])/(math.sqrt(mag_list_1[x])))

            r_doc = max(cos_list)
            r_doc_id = cos_list.index(r_doc)
        #     print(cos_list)
        #     print(r_doc_id)
            file_name= 'input_docs/'+str(r_doc_id)+'.html'
            D = open(file_name,'r')

            doc_html = D.readlines()
            txt = '\nSearch results for :'+s_q+'\n\n'
            for x in doc_html:
                txt = txt+x
            rel_doc = BeautifulSoup(txt).get_text()

            print(rel_doc+'\n')
            print('---------------------------------------------------------------------------------------------------------\n')

        except:
            print('Your search - '+s_q+' - did not match any documents.')
    except:
        print('Your search - '+s_q+' - did not match any documents.')

In [11]:
# interface

In [12]:
text_box = widgets.Text(placeholder= 'search box')
search_button = widgets.Button(description='Search')
results = widgets.Output()

display(text_box,search_button,results)

def onclick(event):
    with results:
        if len(text_box.value)>1:
            cosine_sim(text_box.value)
        else:
            print('No search results for the string')

# def cosine_sim(query):
#     print('search results for ',query)
    
search_button.on_click(onclick)


Text(value='', placeholder='search box')

Button(description='Search', style=ButtonStyle())

Output()